House Price Forecast Project Workflow
1. Define business object
2. Data Exploratory Analysis
   * Univariate
   * Multi-Variate
3. Correlation analysis (pair-wise and attribute combinations)
4. Data cleaning (missing data, outliers, data errors)
   * missing data: fill train dataset with median, and fill test dataset with 0
   * delete some outliers
   * convert some numerical variables to categorical data type (year and month)
5. Data transformation via pipelines (categorical text to number using one hot encoding, feature scaling via normalization/standardization, feature combinations)
  * convert categorical variables using labelencoder( )
  * log-normality for skewed variables
  * robustscaler (sensitive to outliers)
  * feature combinations: combine some variables into one (like garage and basement variables)
6. Train and cross validate 14 models
  * calculated rmse score
  * hyperparameter tuning by GridSerach
  * ensemble learning first stacking meta model
  * ensemble learning second tier stacking meta model with lgboost and xgboost (good for outliers)
7. Evaluate the model with best estimators in the test set

1. Define business object
  * Aim: to forecast the house price in test.csv
  * Target variable: sale price
  * Input variables: 79 variables including sale price in train.csv; 78 variables without sale price in test.csv

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
plt.style.use('ggplot')

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline, make_pipeline
from scipy.stats import skew,norm
from sklearn.decomposition import PCA, KernelPCA
from sklearn.preprocessing import Imputer

In [ ]:
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, cross_val_predict
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import scale
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet, SGDRegressor, BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR, LinearSVR
import lightgbm as lgb
from sklearn.kernel_ridge import KernelRidge
from xgboost import XGBRegressor

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

2. Data Exploratory Analysis

In [ ]:
train

In [ ]:
train.dtypes

In order to better analyzing data, I will combine train and test dataset. Since there may miss a lot of data and may miss different variable value between train and test dataset. I will be very careful dealing with test dataset in data cleaning and data transformation parts.

2.1 Univariate 'SalePrice'

In [ ]:
train['SalePrice'].describe()

There is no missing value for Sale Price and data type is numerical.

In [ ]:
sns.distplot(train['SalePrice'])

In [ ]:
#skewness and kurtosis
print("Skewness: %f" % train['SalePrice'].skew())
print("Kurtosis: %f" % train['SalePrice'].kurt())

* Peak Kurtosis  > 3 (normal distribution kurtosis). 
Positive skewness > 1 (normal distribution skewness).

2.2 Multi-Variate Analysis

In [ ]:
train.describe()

3. Correlation analysis (pair-wise and attribute combinations)

In [ ]:
corrmat = train.corr()
f, ax = plt.subplots(figsize=(20, 9))
sns.heatmap(corrmat, vmax=0.8, square=True)

In [ ]:
k = 11
cols = train.corr().nlargest(k, 'SalePrice')['SalePrice'].index
cm = np.corrcoef(train[cols].values.T)
sns.set(font_scale = 1.25)
plt.figure(figsize = (30,9))
hm = sns.heatmap(cm, cbar=True, annot=True, \
                 square=True, fmt='.2f', annot_kws={'size': 14}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [ ]:
sns.distplot(train['1stFlrSF'])

In [ ]:
sns.distplot(train['GrLivArea'])

Seems like there are some numerical variables also have positive skews. Will do logistic algorithms for those variables later.

4. Data cleaning (outliers,  data errors, missing data)
  * 4.1 outliers

In [ ]:
plt.figure(figsize=(12,6))
plt.scatter(x=train.GrLivArea, y=train.SalePrice)
plt.xlabel("GrLivArea", fontsize=13)
plt.ylabel("SalePrice", fontsize=13)
plt.ylim(0,800000)

The bottom right two points with very large GrLivArea seem like outliers to me (might be large arigucultural land). Also since they are not in the line regression cluster, I will delete them.

In [ ]:
train.drop(train[(train["GrLivArea"]>4000)&(train["SalePrice"]<300000)].index,inplace=True)

In [ ]:
plt.figure(figsize=(12,6))
plt.scatter(x=train.OverallQual, y=train.SalePrice)
plt.xlabel("OverallQual", fontsize=13)
plt.ylabel("SalePrice", fontsize=13)

In [ ]:
train.drop(train[(train['OverallQual']<5) & (train['SalePrice']>200000)].index, inplace=True)

In [ ]:
full=pd.concat([train,test], ignore_index=True)
full.drop(['Id'],axis=1, inplace=True)
full.shape

4.2 data error

In [ ]:
# Some of the non-numeric predictors are stored as numbers; we convert them into strings 
full['MSSubClass'] = full['MSSubClass'].apply(str)
full['YrSold'] = full['YrSold'].astype(str)
full['MoSold'] = full['MoSold'].astype(str)

  * 4.3 Missing Data
   * Here I will spend lots of time on missing data. I will go over with overall missing variables. Then fill missing variables from the most to least. Some categorical features will be transform to dummy variables and some will be deleted. Some numerical will be fill with median or 0. 

In [ ]:
# missing data
total = full.isnull().sum()
total = total[total>0]
percent = (full.isnull().sum()/full.isnull().count())
percent = percent[percent>0]
dtypes = full.dtypes
nulls = np.sum(full.isnull())
dtypes2 = dtypes.loc[(nulls != 0)]
missing_data = pd.concat([total,percent,dtypes2], axis = 1).sort_values(by=0,ascending=False)
missing_data

In [ ]:
# Before getting hands dirty with missing values, I will creat dummy variables for all non-missing categorical features.
full_missing_col = [col for col in full.columns if full[col].isnull().any()]                                  
full_predictors = full.drop(full_missing_col, axis=1)

In [ ]:
full_category_cols= [cname for cname in full_predictors.columns if 
full_predictors[cname].dtype == "object"]
full_category_cols

In [ ]:
# create numerical columns list for future use.
full_num_cols= [cname for cname in full.columns if full[cname].dtype != "object"]
full_num_cols

In [ ]:
full = pd.get_dummies(full,columns=full_category_cols)

It is nice to know in which features miss values in both train and test dataset. I will only fill missing data in the train dataset with "None" or "0", then using get_dummy, without changing test data value.

In [ ]:
for col in ("PoolQC", "MiscFeature", "Alley", "Fence", "FireplaceQu",
           "GarageType", "GarageFinish", "GarageQual", "GarageCond",
           "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1",
            "BsmtFinType2", "MasVnrType"):
    full[col] = full[col].fillna("None")

In [ ]:
for col in ("GarageArea", "GarageCars", "BsmtFinSF1", 
           "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF", "MasVnrArea",
           "BsmtFullBath", "BsmtHalfBath"):
    full[col] = full[col].fillna(0)

Firstly,  "LotFrontage"is missing 14% values. I will fill with the median value group by LotAreaCut. Here I will fill separately in train and test dataset.

In [ ]:
train.groupby(['Neighborhood'])[['LotFrontage']].agg(['mean','median','count'])

In [ ]:
train["LotFrontage"] = train.groupby(["Neighborhood"])[["LotFrontage"]].transform(lambda x: x.fillna(x.median())) 
test["LotFrontage"] =  test.groupby(["Neighborhood"])[['LotFrontage']].transform(lambda x: x.fillna(x.median()))
full["LotFrontage"] = pd.concat([train["LotFrontage"],test["LotFrontage"]],axis =0,ignore_index=True)

In [ ]:
test["LotFrontage"].describe()

In [ ]:
full["LotFrontage"].describe()

In [ ]:
sns.stripplot(x="PoolQC", y="SalePrice", data=full, size = 5, jitter = True)

PoolQC is missing 99% data  and it doesn't show any relation with sale price. Therefore, I will delete PoolQC.

In [ ]:
full.drop('PoolQC', axis=1, inplace=True)

In [ ]:
sns.stripplot(x="MiscFeature", y="SalePrice", data=full, size = 5, jitter = True)

In [ ]:
# There are only a low number of houses in this area with any miscalleanous features.
full.drop(['MiscFeature'],axis=1, inplace=True)

In [ ]:
sns.stripplot(x="Alley", y="SalePrice", data=full, size = 5, jitter = True)

In [ ]:
full['Alley'] = full['Alley'].map({"None":0, "Grvl":1, "Pave":2})
full.head(3)

In [ ]:
sns.stripplot(x="Fence", y="SalePrice", data=full, size = 5, jitter = True)

In [ ]:
full['Fence'] = full['Fence'].map({"None":0, "MnWw":1, "MnPrv":2, "GdPrv":3, "GdWo":4})
full.head(3)

In [ ]:
sns.stripplot(x="FireplaceQu", y="SalePrice", data=full, size = 5, jitter = True)

In [ ]:
# this is a categorical feature with order, I will replace the values by hand.
full['FireplaceQu'] = full['FireplaceQu'].map({"None":0, "Po":1, "Fa":2, "TA":3, "Gd":4, "Ex":5})
full['FireplaceQu'].unique()

Next, I will deal with garage related variables.
  * GarageCond, GarageQual, GarageFinish, and GarageType

In [ ]:
sns.stripplot(x="GarageCond", y="SalePrice", data=full, size = 5, jitter = True)

In [ ]:
full['GarageCond'] = full['GarageCond'].map({"None":0, "Po":1, "Fa":2, "TA":3, "Gd":4, "Ex":5})

In [ ]:
sns.stripplot(x="GarageQual", y="SalePrice", data=full, size = 5, jitter = True)

In [ ]:
full['GarageQual'] = full['GarageQual'].map({"None":0, "Po":1, "Fa":2, "TA":3, "Gd":4, "Ex":5})

In [ ]:
sns.stripplot(x="GarageFinish", y="SalePrice", data=full, size = 5, jitter = True)

In [ ]:
full['GarageFinish'] = full['GarageFinish'].map({"None":0, "Unf":1, "RFn":2, "Fin":3})

In [ ]:
sns.stripplot(x="GarageType", y="SalePrice", data=full, size = 5, jitter = True)

In [ ]:
full['GarageType'] = full['GarageType'].map({"None":0,"2Types":1,"Attchd":2,"Basment":3,"BuiltIn":4,"CarPort":5,"Detchd":6})

Basement related variables: BsmtExposure, BsmtCond, BsmtQual, BsmtFinType2, BsmtFinType1

In [ ]:
sns.stripplot(x="BsmtExposure", y="SalePrice", data=full, size = 5, jitter = True)

In [ ]:
full['BsmtExposure'] = full['BsmtExposure'].map({"None":0, "No":1, "Mn":2, "Av":3, "Gd":4})

In [ ]:
sns.stripplot(x="BsmtCond", y="SalePrice", data=full, size = 5, jitter = True)

In [ ]:
full['BsmtCond'] = full['BsmtCond'].map({"None":0, "Po":1, "Fa":2, "TA":3, "Gd":4, "Ex":5})

In [ ]:
sns.stripplot(x="BsmtQual", y="SalePrice", data=full, size = 5, jitter = True)

In [ ]:
full['BsmtQual'] = full['BsmtQual'].map({"None":0, "Fa":1, "TA":2, "Gd":3, "Ex":4})

In [ ]:
sns.stripplot(x="BsmtFinType2", y="SalePrice", data=full, size = 5, jitter = True)

In [ ]:
full['BsmtFinType2'] = full['BsmtFinType2'].map({"None":0, "Unf":1, "LwQ":2, "Rec":3, "BLQ":4, "ALQ":5,"GLQ":6})

In [ ]:
sns.stripplot(x="BsmtFinType1", y="SalePrice", data=full, size = 5, jitter = True)

In [ ]:
full['BsmtFinType1'] = full['BsmtFinType1'].map({"None":0, "Unf":1, "LwQ":2, "Rec":3, "BLQ":4, "ALQ":5,"GLQ":6})

In [ ]:
sns.stripplot(x="MasVnrType", y="SalePrice", data=full, size = 5, jitter = True)

In [ ]:
full = pd.get_dummies(full, columns = ["MasVnrType"], prefix="MasVnrType")

In [ ]:
sns.stripplot(x="MSZoning", y="SalePrice", data=full, size = 5, jitter = True)

In [ ]:
full = pd.get_dummies(full, columns = ["MSZoning"], prefix="MSZoning")

In [ ]:
plt.subplots(figsize =(15, 5))
plt.subplot(1, 2, 1)
g = sns.countplot(x = "Utilities", data = train).set_title("Utilities - Training")
plt.subplot(1, 2, 2)
g = sns.countplot(x = "Utilities", data = test).set_title("Utilities - Test")

In [ ]:
# Since there is only one category for Utilities feature, therefore I will drop this feature.
full = full.drop(['Utilities'], axis=1)

In [ ]:
# 'GarageYrBlt' is missing 164 values. I assume 'GarageYrBlt' equal to YearBuilt time.Thus these two columns will be highly correlated. Therefore, I delete the 'GarageYrBlt'.
full['GarageYrBlt'] = full['GarageYrBlt'].fillna(full['YearBuilt'])

In [ ]:
for col in ( "GarageArea", "GarageCars", "BsmtFinSF1", 
           "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF", "MasVnrArea",
           "BsmtFullBath", "BsmtHalfBath"):
    full[col] = full[col].fillna(0)

In [ ]:
full['Electrical'] = full['Electrical'].fillna(full['Electrical'].mode()[0])
full['KitchenQual'] = full['KitchenQual'].fillna(full['KitchenQual'].mode()[0])
full['Exterior1st'] = full['Exterior1st'].fillna(full['Exterior1st'].mode()[0])
full['Exterior2nd'] = full['Exterior2nd'].fillna(full['Exterior2nd'].mode()[0])
full['SaleType'] = full['SaleType'].fillna(full['SaleType'].mode()[0])
full["Functional"] = full["Functional"].fillna(full['Functional'].mode()[0])
full = pd.get_dummies(full,columns=['Electrical','KitchenQual','Exterior1st','Exterior2nd','SaleType',"Functional"])

In [ ]:
# check missing data again
total = full.isnull().sum()
total = total[total>0]
percent = (full.isnull().sum()/full.isnull().count())
percent = percent[percent>0]
dtypes = full.dtypes
nulls = np.sum(full.isnull())
dtypes2 = dtypes.loc[(nulls != 0)]
missing_data = pd.concat([total,percent,dtypes2], axis = 1).sort_values(by=0,ascending=False)
missing_data

Great, there is no missing value, except the SalePrice variable in test dataset.

5. Data transformation via pipelines
Feature Engineering
  * 5.1 Convert some numerical variables into categorical variables: LabelEncoder and dummy variables. (Already did during missing value filling)
  * 5.2 Run Box-Cox transformation for skewed numercial variables
  * 5.3 Perform RobustScaler() to better fit outlier values
  * 5.4 PCA (doesn't work well)

In [ ]:
# Adding total sqfootage feature 
full['TotalSF'] = full['TotalBsmtSF'] + full['1stFlrSF'] + full['2ndFlrSF']

In [ ]:
# Check skewness for all numerical variables
skew_features = full[full_num_cols + ['TotalSF']].apply(lambda x: skew(x)).sort_values(ascending=False)
skews = pd.DataFrame({'skew':skew_features})

In [ ]:
# Box-Cox Transformation
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

high_skew = skew_features[abs(skew_features) > 0.75]
skew_index = high_skew.index

for i in skew_index:
    full[i]= boxcox1p(full[i], boxcox_normmax(full[i]+1))

skew_features2 = full[full_num_cols].apply(lambda x: skew(x)).sort_values(ascending=False)
skews2 = pd.DataFrame({'skew':skew_features2})
skews2

In [ ]:
# logistic for SalePrice
plt.subplot(1, 2, 1)
sns.distplot(train.SalePrice, kde=False, fit = norm)

plt.subplot(1, 2, 2)
sns.distplot(np.log(train.SalePrice + 1), kde=False, fit = norm)
plt.xlabel('Log SalePrice')

In [ ]:
train["SalePrice"] = np.log1p(train["SalePrice"])

In [ ]:
full = full.drop(['SalePrice'],axis=1)

In [ ]:
# use robustscaler since maybe there are other outliers.
scaler = RobustScaler()

In [ ]:
n_train=train.shape[0]

X = full[:n_train]
test_X = full[n_train:]
y= train.SalePrice

X_scaled = scaler.fit(X).transform(X)
test_X_scaled = scaler.transform(test_X)

In [ ]:
print(X.shape)
print(test_X.shape)
print (y.shape)

I also tried the PCA to decrease feature number, but the performance does not work well.
  * pca = PCA(n_components=250)
     X_scaled=pca.fit_transform(X_scaled)
     test_X_scaled = pca.transform(test_X_scaled)

5. Machine Learning Modeling 
For this analysis I am trying 14 different algorithms:
 * LinearRegression()
 * Ridge()
 * Lasso(alpha=0.01,max_iter=10000)
 * DecisionTreeRegressor(random_state=1)
 * RandomForestRegressor()
 * GradientBoostingRegressor()
 * SVR()
 * LinearSVR()
 * ElasticNet(alpha=0.001,max_iter=10000)
 * SGDRegressor(max_iter=1000,tol=1e-3)
 * BayesianRidge()
 * KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
 * ExtraTreesRegressor()
 * XGBRegressor()
The method of measuring accuracy was chosen to be Root Mean Squared Error, as described within the competition.

In [ ]:
#Validation function
n_folds = 5
def rmse_cv(model,X,y):
    kf = KFold(n_folds, shuffle=True, random_state=40).get_n_splits(train.values)
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=kf))
    return rmse

In [ ]:
models = [LinearRegression(),Ridge(),Lasso(alpha=0.01,max_iter=10000),DecisionTreeRegressor(random_state=1),RandomForestRegressor(),GradientBoostingRegressor(),SVR(),LinearSVR(),
          ElasticNet(alpha=0.001,max_iter=10000),SGDRegressor(max_iter=1000,tol=1e-3),BayesianRidge(),KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5),
          ExtraTreesRegressor(),XGBRegressor()]

In [ ]:
names = ["LR", "Ridge", "Lasso", "DT","RF", "GBR", "SVR", "LinSVR", "Ela","SGD","Bay","Ker","Extra","Xgb"]
for name, model in zip(names, models):
    score = rmse_cv(model, X_scaled, y)
    print("{}: {:.6f}, {:.4f}".format(name,score.mean(),score.std()))

Hyperparameter Tuning

In [ ]:
class grid():
    def __init__(self,model):
        self.model = model
    
    def grid_get(self,X,y,param_grid):
        grid_search = GridSearchCV(self.model,param_grid,cv=5, scoring="neg_mean_squared_error")
        grid_search.fit(X,y)
        print(grid_search.best_params_, np.sqrt(-grid_search.best_score_))
        grid_search.cv_results_['mean_test_score'] = np.sqrt(-grid_search.cv_results_['mean_test_score'])
        print(pd.DataFrame(grid_search.cv_results_)[['params','mean_test_score','std_test_score']])


In [ ]:
grid(Lasso()).grid_get(X_scaled,y,{'alpha': [.0001, .0003, .0004,.0005, .0006,.0007, .0009, 
          .01, 0.05, 0.1]})

In [ ]:
grid(Ridge()).grid_get(X_scaled,y,{'alpha':[.0001, .0003, .0005, .0007, .0009, 
          .01, 0.05, 0.1, 0.3, 1, 3, 5, 10, 15, 20, 30, 50, 60, 70, 80]})

In [ ]:
grid(Ridge()).grid_get(X_scaled,y,{'alpha':[13,13.1,13.2,13.3,13.4,13.5,13.6,13.7]})

In [ ]:
grid(ElasticNet()).grid_get(X_scaled,y,{'alpha':[0.005,0.0007,0.0008,0.0009],'l1_ratio':[0.5,0.6,0.7,0.8]})

In [ ]:
grid(SVR()).grid_get(X_scaled,y,{'C':[11,13,15],'kernel':["rbf"],"gamma":[0.0003,0.0004],"epsilon":[0.008,0.009]})

In [ ]:
param_grid={'alpha':[0.1,0.15,0.2,0.25,0.3,0.4], 'kernel':["polynomial"], 'degree':[1,2,3,4],'coef0':[0.8,1]}
grid(KernelRidge()).grid_get(X_scaled,y,param_grid)

In [ ]:
import xgboost as xgb
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

In [ ]:
score = rmse_cv(model_xgb,X_scaled,y)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
import lightgbm as lgb
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [ ]:
score = rmse_cv(model_lgb,X_scaled,y)
print("Lgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Ensemble Learning

In [ ]:
class AverageWeight(BaseEstimator, RegressorMixin):
    def __init__(self,mod,weight):
        self.mod = mod
        self.weight = weight
        
    def fit(self,X,y):
        self.models_ = [clone(x) for x in self.mod]
        for model in self.models_:
            model.fit(X,y)
        return self
    
    def predict(self,X):
        w = list()
        pred = np.array([model.predict(X) for model in self.models_])
        # for every data point, single model prediction times weight, then add them together
        for data in range(pred.shape[1]):
            single = [pred[model,data]*weight for model,weight in zip(range(pred.shape[0]),self.weight)]
            w.append(np.sum(single))
        return w

In [ ]:
lasso = Lasso(alpha=0.0005,max_iter=10000)
ridge = Ridge(alpha=13.1)
svr = SVR(gamma= 0.0004,kernel='rbf',C=15,epsilon=0.008)
ker = KernelRidge(alpha=0.1 ,kernel='polynomial',degree=2, coef0=1)
ela = ElasticNet(alpha=0.0007,l1_ratio=0.7,max_iter=10000)
bay = BayesianRidge()

In [ ]:
# assign weights based on their gridsearch score
w1 = 0.3
w2 = 0.05
w3 = 0.15
w4 = 0.1
w5 = 0.3
w6 = 0.1

In [ ]:
weight_avg = AverageWeight(mod = [lasso,ridge,svr,ker,ela,bay],weight=[w1,w2,w3,w4,w5,w6])

In [ ]:
score = rmse_cv(weight_avg,X_scaled,y)
print(score.mean())

In [ ]:
a = Imputer().fit_transform(X_scaled)
b = Imputer().fit_transform(y.values.reshape(-1,1)).ravel()
from mlxtend.regressor import StackingCVRegressor
stack_gen = StackingCVRegressor(regressors=(ridge,ela, 
                                            ker, svr,bay), 
                               meta_regressor=lasso,
                               use_features_in_secondary=False)
stack_gen_model = stack_gen.fit(a, b)
score = rmse_cv(stack_gen_model,a,b)
print(score.mean())

In [ ]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [ ]:
stack_train_pred = stack_gen_model.predict(X_scaled)
c = Imputer().fit_transform(test_X_scaled)
stack_pred = np.expm1(stack_gen_model.predict(c))
print(rmsle(stack_train_pred, y))

In [ ]:
model_xgb.fit(X_scaled, y)
xgb_train_pred = model_xgb.predict(X_scaled)
xgb_pred = np.expm1(model_xgb.predict(test_X_scaled))
print(rmsle(xgb_train_pred, y))

In [ ]:
model_lgb.fit(X_scaled, y)
lgb_train_pred = model_lgb.predict(X_scaled)
lgb_pred = np.expm1(model_lgb.predict(test_X_scaled))
print(rmsle(lgb_train_pred, y))

In [ ]:
ensemble_train = stack_train_pred*0.7 + xgb_train_pred*0.15 + lgb_train_pred*0.15
print(rmsle(ensemble_train, y))

In [ ]:
ensemble = stack_pred*0.8 + xgb_pred*0.1 + lgb_pred*0.1

In [ ]:
my_submission = pd.DataFrame({'Id': test.Id, 'SalePrice': ensemble})
my_submission.to_csv('submission.csv', index=False)